In [ ]:
!pip uninstall numpy -y
!pip install numpy
!pip install tensorflow==2.0.0-beta1 --ignore-installed

### Make sure TensorFlow is version 2.0.0-beta1

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train[:55000]
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2]).T

Y_train = Y_train[:55000]
Y_train = tf.keras.utils.to_categorical(Y_train).T

X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2]).T

print("number of training examples = " + str(X_train.shape[1]))
print("number of test examples = " + str(X_test.shape[1]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

#### TF2 runs in eager execution by default. Disable this to run the TF 1.X code from the post.

In [ ]:
tf.compat.v1.disable_eager_execution()

#### Download TensorFlow  1.X code shown. 
TF2 provides a tility to automatically update code to TF 2.0. Run that upgrade script, print its output, and copy and paste into a cell to run. See post for scripting details

In [ ]:
!curl https://raw.githubusercontent.com/PubChimps/tf2/master/tf1.py >> tf1.py

In [ ]:
### paste upgraded code here
n_input = 784  
n_classes = 10  
dropout = 1  
num_epochs = 60
X = tf.compat.v1.placeholder(tf.float32, [n_input, None])
Y = tf.compat.v1.placeholder(tf.float32, [n_classes, None])
W1 = tf.compat.v1.get_variable("W1", [64,n_input], initializer = tf.compat.v1.random_normal_initializer)
b1 = tf.compat.v1.get_variable("b1", [64,1], initializer = tf.compat.v1.zeros_initializer())
W2 = tf.compat.v1.get_variable("W2", [32,64], initializer = tf.compat.v1.random_normal_initializer)
b2 = tf.compat.v1.get_variable("b2", [32,1], initializer = tf.compat.v1.zeros_initializer())
W3 = tf.compat.v1.get_variable("W3", [10,32], initializer = tf.compat.v1.random_normal_initializer)
b3 = tf.compat.v1.get_variable("b3", [10,1], initializer = tf.compat.v1.zeros_initializer())

Z1 = tf.add(tf.matmul(W1,X), b1)                      
A1 = tf.nn.relu(Z1)                                   
Z2 = tf.add(tf.matmul(W2,A1), b2) 
A2 = tf.nn.relu(Z2) 
Z3 = tf.add(tf.matmul(W3,A2), b3) 

cost = tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=tf.transpose(a=Z3), labels=tf.stop_gradient(tf.transpose(a=Y))))

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=.01).minimize(cost)
init = tf.compat.v1.global_variables_initializer()


with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        _ , epoch_cost = sess.run([optimizer, cost], feed_dict={X: X_train, Y: Y_train})
        if epoch % 20 == 0:
            print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
        
 

    correct_prediction = tf.equal(tf.argmax(input=Z3), tf.argmax(input=Y))
    accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, "float"))
    print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))

In [ ]:
tf.compat.v1.enable_eager_execution()

Once TensorFlow code is run, trying to re-enable eager execution (which is the default runtime in TF 2.0) should throw the error `ValueError: tf.enable_eager_execution must be called at program startup.` 
To continue select 'Kernel' at the top of Watson Studio and Restart. Then skip the cell above and move to `import tensorflow...`

# restart kernel

In [ ]:
import tensorflow as tf
print(tf.__version__)

As in TF 1.X are two ways to run Keras code in TF2.0, eagerly and as a graph, both presented below. The first model trains in Python. Notice how the second, which uses AutoGraph, takes a while to get started running (as TensorFlow converts and optimizes a computaional graph), but then trains much quicker.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test, y_test)

In [ ]:
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.build()
optimizer = tf.keras.optimizers.Adam()
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(y, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    compute_accuracy(y, logits)
    return loss


@tf.function
def train(model, optimizer):
    train_ds = mnist_dataset()
    step = 0
    loss = 0.0
    accuracy = 0.0
    for x, y in train_ds:
        step += 1
        loss = train_one_step(model, optimizer, x, y)
        if tf.equal(step % 10, 0):
            tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
    return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())